In [10]:
import pandas as pd
from pulp import LpMinimize, LpProblem, LpVariable, lpSum, PULP_CBC_CMD, LpStatus, value

# 定义数据
switching_time = [
    [0, 10, 3, 15, 9, 10, 10, 3, 7, 8],
    [12, 0, 10, 16, 4, 17, 16, 15, 19, 9],
    [12, 9, 0, 15, 10, 7, 8, 15, 9, 9],
    [12, 9, 8, 0, 11, 15, 8, 12, 12, 14],
    [8, 7, 11, 23, 0, 18, 18, 11, 15, 16],
    [5, 2, 8, 9, 4, 0, 1, 8, 5, 2],
    [4, 1, 7, 14, 3, 14, 0, 7, 4, 7],
    [10, 7, 13, 14, 6, 18, 18, 0, 4, 7],
    [10, 9, 13, 10, 2, 14, 14, 9, 0, 3],
    [17, 14, 15, 7, 13, 12, 13, 17, 14, 0]
]

packages = {
    1: [3, 6, 4],
    2: [6, 5],
    3: [1, 5],
    4: [9],
    5: [2, 1]
}

num_packages = len(packages)
num_cartridges = 10
slots = 3  # 假设每个包装最多需要3个插槽

# 创建问题实例
model = LpProblem("Minimize_Switching_Times", LpMinimize)

# 定义变量
x = LpVariable.dicts("Use", (range(1, num_packages+1), range(1, slots+1), range(1, num_cartridges+1)), cat='Binary')
y = LpVariable.dicts("Switch", (range(1, num_packages), range(1, slots+1), range(1, num_cartridges+1), range(1, num_cartridges+1)), cat='Binary')

# 目标函数：最小化切换时间
model += lpSum(y[p][s][c][d] * switching_time[c-1][d-1] for p in range(1, num_packages) for s in range(1, slots+1) for c in range(1, num_cartridges+1) for d in range(1, num_cartridges+1))

# 约束条件：每个包装在每个插槽使用一个墨盒
for p in range(1, num_packages+1):
    for s in range(1, slots+1):
        model += lpSum(x[p][s][c] for c in range(1, num_cartridges+1)) == 1

# 约束条件：每个包装必须使用所需的墨盒
for p in range(1, num_packages+1):
    for c in packages[p]:
        model += lpSum(x[p][s][c] for s in range(1, slots+1)) >= 1

# 约束条件：墨盒切换
for p in range(1, num_packages):
    for s in range(1, slots+1):
        for c in range(1, num_cartridges+1):
            for d in range(1, num_cartridges+1):
                model += y[p][s][c][d] >= x[p][s][c] + x[p+1][s][d] - 1

# 求解
model.solve(PULP_CBC_CMD(msg=False))

# 输出结果
print("Status:", LpStatus[model.status])
print("Total Switching Time: ", value(model.objective))

# 输出详细的墨盒切换信息
for p in range(1, num_packages):
    print(f"从包装 {p} 到包装 {p+1} 的墨盒切换情况:")
    for s in range(1, slots+1):
        for c in range(1, num_cartridges+1):
            for d in range(1, num_cartridges+1):
                if y[p][s][c][d].varValue > 0.99:  # 确保只打印实际发生的切换
                    print(f"插槽 {s}：从墨盒 {c} 切换到墨盒 {d}，切换时间: {switching_time[c-1][d-1]}分钟")


Status: Optimal
Total Switching Time:  32.0
从包装 1 到包装 2 的墨盒切换情况:
插槽 1：从墨盒 1 切换到墨盒 1，切换时间: 0分钟
插槽 1：从墨盒 2 切换到墨盒 2，切换时间: 0分钟
插槽 1：从墨盒 3 切换到墨盒 3，切换时间: 0分钟
插槽 1：从墨盒 4 切换到墨盒 4，切换时间: 0分钟
插槽 1：从墨盒 5 切换到墨盒 5，切换时间: 0分钟
插槽 1：从墨盒 6 切换到墨盒 6，切换时间: 0分钟
插槽 1：从墨盒 7 切换到墨盒 7，切换时间: 0分钟
插槽 1：从墨盒 8 切换到墨盒 8，切换时间: 0分钟
插槽 1：从墨盒 9 切换到墨盒 9，切换时间: 0分钟
插槽 1：从墨盒 10 切换到墨盒 10，切换时间: 0分钟
插槽 2：从墨盒 1 切换到墨盒 1，切换时间: 0分钟
插槽 2：从墨盒 2 切换到墨盒 2，切换时间: 0分钟
插槽 2：从墨盒 3 切换到墨盒 3，切换时间: 0分钟
插槽 2：从墨盒 3 切换到墨盒 9，切换时间: 9分钟
插槽 2：从墨盒 4 切换到墨盒 4，切换时间: 0分钟
插槽 2：从墨盒 5 切换到墨盒 5，切换时间: 0分钟
插槽 2：从墨盒 6 切换到墨盒 6，切换时间: 0分钟
插槽 2：从墨盒 7 切换到墨盒 7，切换时间: 0分钟
插槽 2：从墨盒 8 切换到墨盒 8，切换时间: 0分钟
插槽 2：从墨盒 9 切换到墨盒 9，切换时间: 0分钟
插槽 2：从墨盒 10 切换到墨盒 10，切换时间: 0分钟
插槽 3：从墨盒 1 切换到墨盒 1，切换时间: 0分钟
插槽 3：从墨盒 2 切换到墨盒 2，切换时间: 0分钟
插槽 3：从墨盒 3 切换到墨盒 3，切换时间: 0分钟
插槽 3：从墨盒 4 切换到墨盒 4，切换时间: 0分钟
插槽 3：从墨盒 4 切换到墨盒 5，切换时间: 11分钟
插槽 3：从墨盒 5 切换到墨盒 5，切换时间: 0分钟
插槽 3：从墨盒 6 切换到墨盒 6，切换时间: 0分钟
插槽 3：从墨盒 7 切换到墨盒 7，切换时间: 0分钟
插槽 3：从墨盒 8 切换到墨盒 8，切换时间: 0分钟
插槽 3：从墨盒 9 切换到墨盒 9，切换时间: 0分钟
插槽 3：从墨盒 10 切换到墨盒 10，切换时间: 0分钟
